In [1]:
import ROOT


Welcome to JupyROOT 6.30/02


In [3]:
import ROOT

def create_lorentz_vector(pt, eta, phi, energy):
    lv = ROOT.TLorentzVector()
    lv.SetPtEtaPhiE(pt, eta, phi, energy)
    return lv

def process_file(file_path, tree_name):
    file = ROOT.TFile(file_path, "READ")
    tree = file.Get(tree_name)

    invariant_etas = []

    if not tree:
        print(f"Tree not found in file: {file_path}")
    else:
        for event in tree:
            lead_pt = event.lead_pt
            lead_eta = event.lead_eta
            lead_phi = event.lead_phi
            lead_energyRaw = event.lead_energyRaw

            sublead_pt = event.sublead_pt
            sublead_eta = event.sublead_eta
            sublead_phi = event.sublead_phi
            sublead_energyRaw = event.sublead_energyRaw
            
             # Default genweight to 1.0 if not available
            genweight = getattr(event, "genweight", 1.0)

            # Calculate the invariant eta
            lv1 = create_lorentz_vector(lead_pt, lead_eta, lead_phi, lead_energyRaw)
            lv2 = create_lorentz_vector(sublead_pt, sublead_eta, sublead_phi, sublead_energyRaw)
            invariant_eta = (lv1 + lv2).Eta()

            invariant_etas.append(invariant_eta, genweight)

    file.Close()
    return invariant_etas


In [4]:
def process_signal_file(file_path, tree_name, blind_mass_range=False, blind_mass_min=0, blind_mass_max=0):
    file_1 = ROOT.TFile(file_path, "READ")
    tree = file_1.Get(tree_name)

    signal_masses = []

    if not tree:
        print(f"Tree not found in file: {file_path}")
    else:
        for event in tree:
            pt = event.HHbbggCandidate_pt
            eta = event.HHbbggCandidate_eta
            phi = event.HHbbggCandidate_phi
            mass = event.HHbbggCandidate_mass
            genweight = getattr(event, "genweight", 1.0)

            if blind_mass_range and blind_mass_min <= mass <= blind_mass_max:
                continue

            lv_signal = create_lorentz_vector(pt, eta, phi, energy)
            signal_masses.append((lv_signal.Eta(), genweight))

    file_1.Close()
    return signal_masses


In [5]:
data_file_paths = ["../../output_root/Data_EraE.root", "../../output_root/Data_EraF.root", "../../output_root/Data_EraG.root"]

background_files = [
    ("../../output_root/GGJets.root", "GGJets"),
    ("../../output_root/GJetPt20To40.root", "GJetPt20To40"),
    ("../../output_root/GJetPt40.root", "GJetPt40"),
    ("../../output_root/GluGluHToGG.root", "GluGluHToGG"),
    ("../../output_root/ttHToGG.root", "ttHToGG"),
    ("../../output_root/VBFHToGG.root", "VBFHToGG"),
    ("../../output_root/VHToGG.root", "VHToGG")
]

signal_file = "../../output_root/GluGluToHH.root"

cross_sections = {
    "GGJets": 108.3,
    "GJetPt20To40": 242.5,
    "GJetPt40": 919.1,
    "GluGluHToGG": 39.91,
    "ttHToGG": 0.5687,
    "VBFHToGG": 4.359,
    "VHToGG": 2.943,
    "GluGluToHH": 0.0879
}

tree_name = "DiphotonTree/data_125_13TeV_NOTAG"

integrated_luminosities = {
    "Data_EraE": 5.8070,
    "Data_EraF": 17.7819,
    "Data_EraG": 3.0828
}

In [ ]:
blind_mass = True  # Set this to True to enable blinding

data_invariant_masses = []
signal_masses = []

for data_file_path in data_file_paths:
    data_sample_name = data_file_path.split("/")[-1].split(".")[0]
    luminosity = integrated_luminosities.get(data_sample_name, 1.0)
    data_invariant_masses.extend(process_file(data_file_path, tree_name, blind_mass))

    file = ROOT.TFile(data_file_path, "READ")
    tree = file.Get(tree_name)
    if tree:
        for event in tree:
            pt = event.HHbbggCandidate_pt
            eta = event.HHbbggCandidate_eta
            phi = event.HHbbggCandidate_phi
            mass = event.HHbbggCandidate_mass
            genweight = getattr(event, "genweight", 1.0)  # Default genweight to 1.0 if not available

            # Apply blinding
            if blind_mass and 110 <= mass <= 140:
                continue

            lv_signal = create_lorentz_vector(pt, eta, phi, mass)
            signal_masses.append((lv_signal.M(), genweight))
    file.Close()
